In [1]:
#Importing BeautifulSoup, Pandas and other required modules
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import numpy as np
import datetime
import time

In [2]:
#Importing URLs from PC_World_Urls.pickle created by PC_World_Scraper_FindProductURLs.ipynb
urls = pd.read_pickle('PC_World_Urls.pickle')
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
shop_web_addr = 'https://www.pcworld.co.uk'

In [3]:
#Showing 5 first URLs
urls[:5]

['https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/hp-14-bp059sa-14-laptop-jet-black-10166575-pdt.html',
 'https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/thomson-neo10-10-1-laptop-white-10175040-pdt.html',
 'https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/hp-14-bp056sa-14-laptop-snow-white-10166801-pdt.html',
 'https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/hp-notebook-15-bw054sa-15-6-laptop-grey-10164662-pdt.html',
 'https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/acer-cb3-431-14-full-hd-chromebook-gold-10148917-pdt.html']

In [4]:
#Creating first Pandas DataFrame from all laptops listed in first page result

techDetails = pd.DataFrame()
url = urls[0]
url='https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/lenovo-ideapad-320s-14ikb-14-laptop-grey-10169166-pdt.html'
page = requests.get(url, headers=headers)
print(url)

while page.status_code == 503:
    #print('Service Unavailable "Response Code 503"')
    page = requests.get(url, headers=headers)

page_html = page.text

# Html Parsing
page_soup = soup(page_html, 'html.parser')
#print(page_soup.h1)

#Extracting SKU, Brand, Model, Price and Original Price from the product page
sku = page_soup.findAll('p', {'class':'prd-code'})[0].text.split()[-1]
product_header = page_soup.findAll('h1', {'class':'page-title nosp'})[0]
product_description = product_header.findAll('span')[0].text + ' ' + product_header.findAll('span')[1].text
brand = product_description.split()[0]
x=1
if brand in ['Fujitus', 'Fujtisu']:
    brand = 'Fujitsu'
if brand in ['Refurbished', 'New', 'Pre-Owned']:
    brand = product_description.split()[1]
    x=2
if brand in ['Hewlett']:
    brand = product_description.split()[2]
    x=3
    if brand in ['Refurbished', 'New', 'Pre-Owned']:
        brand = product_description.split()[3]
        x=4 
if brand in ['GRADE', 'Box']:
    brand = product_description.split()[3]
    x=4
    if brand in ['Refurbished', 'New', 'Pre-Owned']:
        brand = product_description.split()[4]
        x=5
model=product_description.split()[x:x+2]
model = [x for x in model if x not in ['Core', '14"', '15"', '15.6"', '10.1"', '12"', '13"', '11.6"', '17.3"']]

product_action=page_soup.findAll('div', {'id':'product-actions'})[0]

try:
    price = product_action.findAll('strong', {'class':'current'})[0].text.strip().split('£')[-1]
except:
    price='Out of Stock'
try:
    original_price = product_action.findAll('div', {'class':'prd-past-amounts'})[0].span.text.split('£')[-1]
except:
    original_price = price
    
currency = 'GBP'

# Grabbing Spec Data from Tab2 page on
specs = page_soup.find('div', {'id':'tab2'})

#Finding Labels and Values from the Spec and adding the Model, Brand Etc to the collected Labels
Labels = specs.findAll('th')
Labels = [ x.text.replace(',', ';') for x in Labels ]
Labels = ['retailer', 'timestamp', 'brand', 'model', 'price', 'currency', 'original_price'] + Labels + ['product_description', 'url']

Values = specs.findAll('td')
Values = [ x.text.replace(',', ';') for x in Values ]
ts = time.time()
dt = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
Values = ['LaptopsDirect', dt, brand, ' '.join(model), price, currency, original_price] + Values + [product_description, url]

#Creating Dataframe from the Labels and Values Lists
techDetails = pd.DataFrame({'Labels': Labels,sku: Values})

#Rearranging Columns if needed
if techDetails.columns[0] != 'Labels':
    techDetails=techDetails[[techDetails.columns[1],techDetails.columns[0]]]
print(techDetails.head())

https://www.pcworld.co.uk/gbuk/computing/laptops/laptops/lenovo-ideapad-320s-14ikb-14-laptop-grey-10169166-pdt.html
      Labels               215754
0   retailer        LaptopsDirect
1  timestamp  2018-01-09 19:14:35
2      brand               LENOVO
3      model   IdeaPad 320s-14IKB
4      price               299.00


In [5]:
# Run same routine for all th eother product pages
for url in urls[1:]:
    page = requests.get(url, headers=headers)
    #print(url)

    while page.status_code == 503:
        #print('Service Unavailable "Response Code 503"')
        page = requests.get(url, headers=headers)

    page_html = page.text

    # Html Parsing
    page_soup = soup(page_html, 'html.parser')
    #print(page_soup.h1)

    sku = page_soup.findAll('p', {'class':'prd-code'})[0].text.split()[-1]
    product_header = page_soup.findAll('h1', {'class':'page-title nosp'})[0]
    product_description = product_header.findAll('span')[0].text + ' ' + product_header.findAll('span')[1].text
    brand = product_description.split()[0]
    x=1
    if brand in ['Fujitus', 'Fujtisu']:
        brand = 'Fujitsu'
    if brand in ['Refurbished', 'New', 'Pre-Owned']:
        brand = product_description.split()[1]
        x=2
    if brand in ['Hewlett']:
        brand = product_description.split()[2]
        x=3
        if brand in ['Refurbished', 'New', 'Pre-Owned']:
            brand = product_description.split()[3]
            x=4 
    if brand in ['GRADE', 'Box']:
        brand = product_description.split()[3]
        x=4
        if brand in ['Refurbished', 'New', 'Pre-Owned']:
            brand = product_description.split()[4]
            x=5
    model=product_description.split()[x:x+2]
    model = [x for x in model if x not in ['Core', '14"', '15"', '15.6"', '10.1"', '12"', '13"', '11.6"', '17.3"']]

    product_action=page_soup.findAll('div', {'id':'product-actions'})[0]
    
    try:
        price = product_action.findAll('strong', {'class':'current'})[0].text.strip().split('£')[-1]
    except:
        price='Out of Stock'
    try:
        original_price = product_action.findAll('div', {'class':'prd-past-amounts'})[0].span.text.split('£')[-1]
    except:
        original_price = price
        
    currency = 'GBP'
    specs = page_soup.find('div', {'id':'tab2'})
    
    try:
        Labels = specs.findAll('th')
        Labels = [ x.text.replace(',', ';') for x in Labels ]
        Labels = ['retailer', 'timestamp', 'brand', 'model', 'price', 'currency', 'original_price'] + Labels + ['product_description', 'url']

        Values = specs.findAll('td')
        Values = [ x.text.replace(',', ';') for x in Values ]
        ts = time.time()
        dt = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
        Values = ['PC_World', dt, brand, ' '.join(model), price, currency, original_price] + Values + [product_description, url]
        
        techDetail = pd.DataFrame({'Labels': Labels,sku: Values})
        
        if techDetail.columns[0] != 'Labels':
            techDetail=techDetail[[techDetail.columns[1],techDetail.columns[0]]]
        
        techDetails = pd.merge(techDetails, techDetail, on='Labels', how='outer')
        techDetails = techDetails.drop_duplicates()
    except:
        print(sku)
        print('ERROR on URL:',url)

techDetails = techDetails.set_index('Labels')

techDetails = techDetails.transpose()
techDetails.index.rename('SKU', inplace=True)
print(techDetails.head())

Labels         retailer            timestamp    brand                model  \
SKU                                                                          
215754    LaptopsDirect  2018-01-09 19:14:35   LENOVO   IdeaPad 320s-14IKB   
229200_x       PC_World  2018-01-09 19:14:35  THOMSON                NEO10   
212388_x       PC_World  2018-01-09 19:14:36       HP           14-bp056sa   
201802_x       PC_World  2018-01-09 19:14:37       HP  Notebook 15-bw054sa   
169493_x       PC_World  2018-01-09 19:14:38     ACER              CB3-431   

Labels     price currency original_price        Type     Operating system  \
SKU                                                                         
215754    299.00      GBP         529.99      Laptop           Windows 10   
229200_x  129.99      GBP         129.99      Laptop  Windows 10 (64-bit)   
212388_x  219.99      GBP         299.99      Laptop  Windows 10 (64-bit)   
201802_x  299.99      GBP         399.99      Laptop  Windows 10 (64

In [6]:
#Save Date in Pickle and CSV format
techDetails.to_pickle('PC_World_Latops_Scrape.pickle')
techDetails.to_csv('PC_World_Latops_Scrape.csv')